In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/PyAI'

/content/drive/MyDrive/PyAI


In [3]:
import torchvision.datasets as datasets #torch에서 데이터 불러오기
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [4]:
train = datasets.CIFAR10(root = "/data", train = True, transform=transforms.ToTensor(), download=True)
test = datasets.CIFAR10(root = "/data", train = False, transform=transforms.ToTensor(), download=True)

train_data = DataLoader(train, batch_size = 100, shuffle=True)
test_data = DataLoader(test, batch_size=1000, shuffle = False)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#

#
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3, device = device), #(3,32,32)>(32,30,30)
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3, device = device), #(32,30,30)>(32,28,28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32,28,28)>(16,14,14)
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3, device = device), #(32,14,14)>(64,12,12)
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3, device = device), #(64,12,12)>(64,10,10)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64,10,10)>(64,5,5)
            nn.Dropout(0.25),

            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3, device = device), #(64,5,5)>(128,3,3)
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3, device = device), #(128,3,3)>(256,1,1)
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.g = nn.Sequential(
            nn.Linear(256, 128, device = device),
            nn.ReLU(),
            nn.Linear(128, 64, device = device),
            nn.ReLU(),
            nn.Linear(64, 10, device = device),
        )
    def forward(self, x) :
        if len(x.shape) == 3 :
            x = x.unsqueeze(dim = 0)
        x = self.f(x)
        x = x.reshape(x.shape[0], -1)
        x = self.g(x)
        return x

F = NN()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr = 0.001)
epoch = 100
F.train()

for e in range(epoch) :
    loss_sum = 0
    for x, t in train_data :
        x = x.to(device)
        t = t.to(device)
#
        y = F(x)
#
        loss = loss_func(y, t)
        loss_sum += loss
#
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"epoch {e+1} | loss {loss_sum / len(train_data)}")

epoch 1 | loss 1.8986808061599731
epoch 2 | loss 1.5129705667495728
epoch 3 | loss 1.3249338865280151
epoch 4 | loss 1.210802674293518
epoch 5 | loss 1.1266666650772095
epoch 6 | loss 1.0610690116882324
epoch 7 | loss 1.008978247642517
epoch 8 | loss 0.964852511882782
epoch 9 | loss 0.9192347526550293
epoch 10 | loss 0.8908122181892395
epoch 11 | loss 0.8645144701004028
epoch 12 | loss 0.8429259657859802
epoch 13 | loss 0.8184678554534912
epoch 14 | loss 0.8006001710891724
epoch 15 | loss 0.7842457294464111
epoch 16 | loss 0.7715557813644409
epoch 17 | loss 0.7597318887710571
epoch 18 | loss 0.7469639778137207
epoch 19 | loss 0.7307722568511963
epoch 20 | loss 0.7260173559188843
epoch 21 | loss 0.7085958123207092
epoch 22 | loss 0.712296724319458
epoch 23 | loss 0.6993889808654785
epoch 24 | loss 0.692719578742981
epoch 25 | loss 0.6779728531837463
epoch 26 | loss 0.6774918437004089
epoch 27 | loss 0.6653558015823364
epoch 28 | loss 0.6587547063827515
epoch 29 | loss 0.656877875328064


In [11]:
cnt = 0
F.eval()

for x, t in test_data :
    x = x.to(device)
    t = t.to(device)
    y = F(x)
    for i in range(len(x)) :
        if torch.argmax(y[i]) == t[i] :
            cnt += 1

print(cnt / len(test))

0.8084


In [12]:
torch.save(F, "CIFAR10.pt")

In [8]:
print(train[0][0].shape) #x의 모양
print(min(train.targets)) #t최댓값
print(max(train.targets)) #t최솟값
print(train[0][0])

torch.Size([3, 32, 32])
0
9
tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
         [0.0824, 0.0000, 0.03